### 텍스트 조작 예제 : 고급편

### 학습 내용
* Completion과 Edit 메서드 연결
* 회사이름 “Apple” vs 과일이름 "Apple"(문맥 스터핑)
* 사용자 정의 구조를 기반으로 암호 화폐 정보 얻기(문맥 스터핑)
* 리눅스 명령을 사용하는 챗봇 어시스턴트 만들기

### 사전 준비
 * 구글 코랩 환경은 일정 시간이후에 초기화가 되기 때문에 두가지 작업을 매번 수행해야 함.
   * chatgpt.env 파일 생성이 필요.
     * 준비된 chatgpt.env를 내용을 변경하여 업로드 하거나 또는 API_KEY와 ORG_ID를 확인하여 생성한다.
   * pip install openai 설치

In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.7 MB/s eta 0:00:00


### Completion과 Edit 메서드 연결

In [3]:
import os
import openai

def init_api():
    with open("chatgpt.env") as env:
        for line in env:
            key, value = line.strip().split("=")
            os.environ[key] = value

    openai.api_key = os.environ.get("API_KEY")
    openai.organization = os.environ.get("ORG_ID")

init_api()

prompt = "The first programming language to be invented was Plankalkül, which was de\
signed by Konrad Zuse in the 1940s, but not publicly known until 1972 (and not imple\
mented until 1998).   The first widely known and successful high-level programming l\
anguage was Fortran,   developed from 1954 to 1957 by a team of IBM researchers led \
by John Backus.   The success of FORTRAN led to the formation of a committee of scie\
ntists to develop a universal computer language;   the result of their effort was AL\
GOL 58.  Separately, John McCarthy of MIT developed Lisp,  the first language with o\
rigins in academia to be successful. With the success of these initial efforts, prog\
ramming languages became an active topic of research in the 1960s and beyond\n\nTwee\
t with hashtags:"

# 01 트웟 생성
english_tweet = openai.Completion.create(
  model="text-davinci-002",
  prompt=prompt,
  temperature=0.5,
  max_tokens=20
)

english_tweet_text = english_tweet["choices"][0]["text"].strip()
print("English Tweet:")
print(english_tweet_text)

# 02 생성된 내용 번역 요청
spanish_tweet = openai.Edit.create(
  model="text-davinci-edit-001",
  input=english_tweet_text,
  instruction="Translate to Spanish",
  temperature=0.5
)

spanish_tweet_text = spanish_tweet["choices"][0]["text"].strip()
print("Spanish Tweet:")
print(spanish_tweet_text)


English Tweet:
The #first #programming #language to be invented was #Plankalkül
Spanish Tweet:
El #primer #lenguaje de #programación en ser inventado fue el #Plankalkül


### 회사이름 “Apple” vs 과일이름 "Apple" (문맥 스터핑)

#### 단어 'light'의 품사를 결정하기

In [4]:
prompt = "Determine the part of speech of the word 'light'.\n\n"

result = openai.Completion.create(
    model="text-davinci-002",
    prompt=prompt,
    max_tokens=20,
    temperature=1
)

print(result.choices[0]["text"].strip())


The word 'light' is a noun.


#### 힌트(문장)을 주고, light 품사 물어보기

In [5]:
prompt_a = "The light is red. Determine the part of speech of the word 'light'.\n\n"
prompt_b = "This desk is very light. Determine the part of speech of the word 'light'.\n\n"
prompt_c = "You light up my life. Determine the part of speech of the word 'light'.\\n\n"

for prompt in [prompt_a, prompt_b, prompt_c]:
  result = openai.Completion.create(
      model="text-davinci-002",
      prompt=prompt,
      max_tokens=20,
      temperature=0
  )

  print(result.choices[0]["text"].strip())


The word 'light' is a noun.
adjective
verb


#### 하나의 의미는 회사, 하나의 의미는 과일

In [6]:
prompt1 = "Huawei:\ncompany\n\nGoogle:\ncompany\n\nMicrosoft:\ncompany\n\nApple:\n"
prompt2 = "Huawei:\ncompany\n\nGoogle:\ncompany\n\nMicrosoft:\ncompany\n\nApricot:\nFruit\n\nApple:\n"

result = openai.Completion.create(
    model="text-davinci-002",
    prompt=prompt1,
    max_tokens=20,
    temperature=0,
    stop=["\n", " "]
)

print(result.choices[0]["text"].strip())


company


In [7]:
prompt1 = "Huawei:\ncompany\n\nGoogle:\ncompany\n\nMicrosoft:\ncompany\n\nApple:\n"
prompt2 = "Huawei:\ncompany\n\nGoogle:\ncompany\n\nMicrosoft:\ncompany\n\nApricot:\nFruit\n\nApple:\n"

result = openai.Completion.create(
    model="text-davinci-002",
    prompt=prompt2,
    max_tokens=20,
    temperature=0,
    stop=["\n", " "]
)

print(result.choices[0]["text"].strip())


Fruit


### 사용자 정의 구조를 기반으로 암호 화폐 정보 가져오기(문맥 스터핑)

In [13]:
import os
import openai

def init_api():
    with open("chatgpt.env") as env:
        for line in env:
            key, value = line.strip().split("=")
            os.environ[key] = value

    openai.api_key = os.environ.get("API_KEY")
    openai.organization = os.environ.get("ORG_ID")

init_api()

prompt = """Input: Bitcoin
Output:
BTC was created in 2008, you can learn more about it here: https://bitcoin.org/en/ a\
nd get the latest price here: https://www.coingecko.com/en/coins/bitcoin.
It's all-time high is $64,895.00 and it's all-time low is $67.81.

Input: Ethereum
Output:
ETH was created in 2015, you can learn more about it here: https://ethereum.org/en/ \
and get the latest price here: https://www.coingecko.com/en/coins/ethereum
It's all-time high is $4,379.00 and it's all-time low is $0.43.

Input: Dogecoin
Output:
DOGE was created in 2013, you can learn more about it here: https://dogecoin.com/ an\
d get the latest price here: https://www.coingecko.com/en/coins/dogecoin
It's all-time high is $0.73 and it's all-time low is $0.000002.

Input: Cardano
Output:\n"""

result = openai.Completion.create(
    model="text-davinci-002",
    prompt=prompt,
    max_tokens=200,
    temperature=0
)

print(result.choices[0]["text"].strip())


ADA was created in 2017, you can learn more about it here: https://www.cardano.org/en/home/ and get the latest price here: https://www.coingecko.com/en/coins/cardano
It's all-time high is $1.33 and it's all-time low is $0.000017.


#### prompt에 출력 형식 변경해 보기 - HTML추가

In [14]:
import os
import openai

def init_api():
    with open("chatgpt.env") as env:
        for line in env:
            key, value = line.strip().split("=")
            os.environ[key] = value

    openai.api_key = os.environ.get("API_KEY")
    openai.organization = os.environ.get("ORG_ID")

init_api()

prompt = """Input: Bitcoin
Output:
BTC was created in 2008, you can learn more about it <ahref="https://bitcoin.org/en/">here</a> and get the latest price <a href="https://www.coingecko.com/en/coins/bitcoin">here</a>.
It's all-time high is $64,895.00 and it's all-time low is $67.81.

Input: Cardano
Output:\n"""

result = openai.Completion.create(
    model="text-davinci-002",
    prompt=prompt,
    max_tokens=200,
    temperature=0
)

print(result.choices[0]["text"].strip())


Cardano was created in 2015, you can learn more about it <a href="https://www.cardano.org/en/home/">here</a> and get the latest price <a href="https://www.coingecko.com/en/coins/cardano">here</a>.
It's all-time high is $1.33 and it's all-time low is $0.00004602.


### 리눅스 명령어 도움말을 위한 챗봇 어시스턴트 만들기

In [15]:
import os
import openai

def init_api():
    with open("chatgpt.env") as env:
        for line in env:
            key, value = line.strip().split("=")
            os.environ[key] = value

    openai.api_key = os.environ.get("API_KEY")
    openai.organization = os.environ.get("ORG_ID")

init_api()

prompt = """
Input: List all the files in the current directory
Output: ls -l

Input: List all the files in the current directory, including hidden files
Output: ls -la

Input: Delete all the files in the current directory
Output: rm *

Input: Count the number of occurrences of the word "sun" in the file "test.txt"
Output: grep -o "sun" test.txt | wc -l

Input:{}
Output:
"""

result = openai.Completion.create(
    model="text-davinci-002",
    prompt=prompt.format("Count the number of files in the current directory"),
    max_tokens=200,
    temperature=0
)

print(result.choices[0]["text"].strip())


ls -l | wc -l


#### click 패키지 설치

In [2]:
!pip install click==8.1.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.1.7
    Uninstalling click-8.1.7:
      Successfully uninstalled click-8.1.7


In [16]:
import os
import openai
import click

def init_api():
    with open("chatgpt.env") as env:
        for line in env:
            key, value = line.strip().split("=")
            os.environ[key] = value

    openai.api_key = os.environ.get("API_KEY")
    openai.organization = os.environ.get("ORG_ID")

init_api()

_prompt = """
Input: List all the files in the current directory
Output: ls -l

Input: List all the files in the current directory, including hidden files
Output: ls -la

Input: Delete all the files in the current directory
Output: rm *

Input: Count the number of occurrences of the word "sun" in the file "test.txt"
Output: grep -o "sun" test.txt | wc -l

Input: {}
Output:"""

while True:
    request = input(click.style("Input:", fg="green"))
    prompt = _prompt.format(request)
    result = openai.Completion.create(
        model="text-davinci-002",
        prompt=prompt,
        temperature=0.0,
        max_tokens=100,
        stop=["\n"]
    )

    command = result.choices[0].text.strip()
    click.echo(click.style("Output: ", fg="yellow") + command)
    click.echo()


Input:list all
Output: ls

Input:delete all
Output: rm -r *

Input:count all files
Output: ls -l | wc -l

Input:count all directories
Output: find . -type d | wc -l

Input:count all files that are not directories
Output: find . ! -type d | wc -l



KeyboardInterrupt: ignored

#### 무한루프로 인해 종료 명령을 추가해 보기

In [17]:
import os
import openai
import click

def init_api():
    with open("chatgpt.env") as env:
        for line in env:
            key, value = line.strip().split("=")
            os.environ[key] = value
        openai.api_key = os.environ.get("API_KEY")
        openai.organization = os.environ.get("ORG_ID")

init_api()

_prompt = """
Input: List all the files in the current directory
Output: ls -l
Input: List all the files in the current directory, including hidden files
Output: ls -la
Input: Delete all the files in the current directory
Output: rm *
Input: Count the number of occurrences of the word "sun" in the file "test.txt"
Output: grep -o "sun" test.txt | wc -l
Input: Create a new file. "test.txt"
Output: touch file1.txt
Input: {}
Output:"""

while True:
    request = input(click.style("Input (type 'exit' to quit): ", fg="green"))
    prompt = _prompt.format(request)
    if request == "exit":
        break
    result = openai.Completion.create(
        model="text-davinci-002",
        prompt=prompt,
        temperature=0.0,
        max_tokens=100,
        stop=["\n"]
    )
    command = result.choices[0].text.strip()
    click.echo(click.style("Output: ", fg="yellow") + command)
    click.echo()


Input (type 'exit' to quit): create file
Output: touch file1.txt

Input (type 'exit' to quit): list all
Output: ls -l

Input (type 'exit' to quit): delete all
Output: rm -rf *

Input (type 'exit' to quit): count all directories
Output: find . -type d | wc -l

Input (type 'exit' to quit): exit


#### ChatGPT가 생성해준 리눅스 명령을 실행하는 프로그램 만들어보기

In [ ]:
import os
import openai
import click

def init_api():
    with open("chatgpt.env") as env:
        for line in env:
            key, value = line.strip().split("=")
            os.environ[key] = value
        openai.api_key = os.environ.get("API_KEY")
        openai.organization = os.environ.get("ORG_ID")

init_api()

_prompt = """
Input: List all the files in the current directory
Output: ls -l
Input: List all the files in the current directory, including hidden files
Output: ls -la
Input: Delete all the files in the current directory
Output: rm *
Input: Count the number of occurrences of the word "sun" in the file "test.txt"
Output: grep -o "sun" test.txt | wc -l
Input: Create a new file. "test.txt"
Output: touch file1.txt
Input: {} Output:"""

while True:
    request = input(click.style("Input (type 'exit' to quit): ", fg="green"))
    if request == "exit":
        break
    prompt = _prompt.format(request)
    result = openai.Completion.create(
        model="text-davinci-002",
        prompt=prompt,
        temperature=0.0,
        max_tokens=100,
        stop=["\n"]
    )
    command = result.choices[0].text.strip()
    click.echo(click.style("Output: ", fg="yellow") + command)
    click.echo(click.style("Execute? (y/n): ", fg="yellow"), nl=False)
    choice = input()
    if choice == "y":
        os.system(command)
    elif choice == "n":
        continue
    else:
        click.echo(click.style("Invalid choice. Please enter 'y' or 'n'.", fg="red"))
    click.echo()


Input (type 'exit' to quit): create file
Output: touch file1.txt
Execute? (y/n): y

Input (type 'exit' to quit): list all files in /tmp
Output: ls /tmp
Execute? (y/n): y

Input (type 'exit' to quit): exit


#### try..except 추가하기

In [ ]:
import os
import openai
import click

def init_api():
    with open("chatgpt.env") as env:
        for line in env:
            key, value = line.strip().split("=")
            os.environ[key] = value
        openai.api_key = os.environ.get("API_KEY")
        openai.organization = os.environ.get("ORG_ID")

init_api()

_prompt = """
Input: List all the files in the current directory
Output: ls -l
Input: List all the files in the current directory, including hidden files
Output: ls -la
Input: Delete all the files in the current directory
Output: rm *
Input: Create a new file. "test.txt"
Output: touch file1.txt
Input: {} Output:"""

while True:
    request = input(click.style("Input (type 'exit' to quit): ", fg="green"))
    if request == "exit":
        break
    prompt = _prompt.format(request)
    try:
        result = openai.Completion.create(
            model="text-davinci-002",
            prompt=prompt,
            temperature=0.0,
            max_tokens=100,
            stop=["\n"]
        )
        command = result.choices[0].text.strip()
        click.echo(click.style("Output: ", fg="yellow") + command)
        click.echo(click.style("Execute? (y/n): ", fg="yellow"), nl=False)
        choice = input()
        if choice == "y":
            os.system(command)
        elif choice == "n":
            continue
        else:
            click.echo(click.style("Invalid choice. Please enter 'y' or 'n'.", fg="red"))
    except Exception as e:
        click.echo(click.style("The command could not be executed. {} ".format(e), fg="red"))
        pass
    click.echo()


Input (type 'exit' to quit): create a new file
Output: touch file1.txt
Execute? (y/n): y

Input (type 'exit' to quit): exit
